<a href="https://colab.research.google.com/github/jerpint/voicemd/blob/master/voicemd/colab/Manual_Age_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ## Initialization + Evaluation
#@markdown Hit the play button on the left to initialize this notebook.
#@markdown Listen to each sound file and give it an age bracket.
#@markdown We recommend doing this with headphones.
#@markdown Do as many as you can (you don't have to do them all). 
#@markdown When done, go to the bottom to evaluate your results.
#@markdown Save a copy of your results and send it back to the evaluator.
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Audio

print('initializing...')
if not os.path.isdir('cv_balanced/'):
  print("Downloading data...")
  !curl -L -o cv_balanced.zip "https://www.dropbox.com/s/uee4r9s3w7imj9k/cv_balanced.zip?dl=1"
  !unzip -o -q cv_balanced.zip
print('done!')

total_samples = 100
metadata = pd.read_csv('cv_balanced/cv_metadata_balanced.csv')
shuffled_metadata = metadata.sample(n=len(metadata), random_state=1337)

age_categories=['twenties', 'thirties', 'fourties', 'fifties', 'sixties']
assert set(age_categories) == set(shuffled_metadata.age.unique())
filenames = shuffled_metadata['path'].iloc[0:total_samples]
all_widgets = {}

for i, fname in enumerate(filenames):
  print(f'{i+1}. \n')
  current_widget =  widgets.RadioButtons(
        options=age_categories, 
        description='Choose Age:', 
        value=None,
        disabled=False,
  )
  all_widgets[fname] = {}
  all_widgets[fname]['widget'] = current_widget
  all_widgets[fname]['gt'] = shuffled_metadata[shuffled_metadata.path == fname]['age'].values[0]
  display(Audio('cv_balanced/clips/' + fname))
  display(current_widget)
  print("\n"*4)

In [ ]:
#@title ## Results
#@markdown Hit the play button on this cell to compute your stats. 
#@markdown Copy-paste and send your results to your evaluator.

from sklearn.metrics import confusion_matrix, accuracy_score

# gather results
y_true = []
y_pred = []
for fname in all_widgets:
  if all_widgets[fname]['widget'].value is None:
    continue
  y_true.append(all_widgets[fname]['gt'])
  y_pred.append(all_widgets[fname]['widget'].value)

print("Number of samples analyzed:", len(y_pred))
print("Confusion Matrix:")
display(confusion_matrix(y_true, y_pred, labels=age_categories))
print("\n")
print(f"Accuracy: {accuracy_score(y_true, y_pred)*100} %")
